In [32]:
import time

from langchain_google_genai.llms import GoogleGenerativeAI
from langchain_core.prompts import PromptTemplate

import pandas as pd

from dotenv import load_dotenv

import os

from tqdm import tqdm

from IPython.display import display, Markdown

In [13]:
load_dotenv()

True

In [5]:
quotes = pd.read_json("cleaned_quotes.json", lines=True)
quotes

,0,1,2,3,4,5,6,7,8,9,...,2984,2985,2986,2987,2988,2989,2990,2991,2992,2993
0,"All that is gold does not glitter,\nNot all th...",Not all those who wander are lost.\n,"I wish it need not have happened in my time,"" ...",I don't know half of you half as well as I sho...,All we have to decide is what to do with the t...,If more of us valued food and cheer and song a...,Never laugh at live dragons.\n,Faithless is he that says farewell when the ro...,"The world is indeed full of peril, and in it t...",Deserves it! I daresay he does. Many that live...,...,"Where did you go to, if I may ask?' said Thori...","In this hour, I do not believe that any darkne...",Oft hope is born when all is forlorn.\n,The wide world is all about you: you can fence...,I do not love the bright sword for its sharpne...,He that breaks a thing to find out what it is ...,"Home is behind, the world ahead,\nAnd there ar...","There is more in you of good than you know, ch...","Good Morning!"" said Bilbo, and he meant it. Th...","In sorrow we must go, but not in despair. Beho..."


In [6]:
len(quotes), len(quotes.columns), type(quotes)

(1, 2994, pandas.core.frame.DataFrame)

In [7]:
# transpose the columns
quotes = quotes.T
quotes

,0
0,"All that is gold does not glitter,\nNot all th..."
1,Not all those who wander are lost.\n
2,"I wish it need not have happened in my time,"" ..."
3,I don't know half of you half as well as I sho...
4,All we have to decide is what to do with the t...
...,...
2989,He that breaks a thing to find out what it is ...
2990,"Home is behind, the world ahead,\nAnd there ar..."
2991,"There is more in you of good than you know, ch..."
2992,"Good Morning!"" said Bilbo, and he meant it. Th..."


In [8]:
len(quotes), len(quotes.columns), type(quotes)

(2994, 1, pandas.core.frame.DataFrame)

In [9]:
prompt_template = PromptTemplate.from_template(
    "Summarize the following quote in simple English in a paragraph: '{quote}'")

In [35]:
# indexing in [column][row] format
prompt_template.invoke(quotes[0][0])

StringPromptValue(text="Summarize the following quote in simple English in a paragraph: 'All that is gold does not glitter,\nNot all those who wander are lost;\nThe old that is strong does not wither,\nDeep roots are not reached by the frost.\n\nFrom the ashes a fire shall be woken,\nA light from the shadows shall spring;\nRenewed shall be blade that was broken,\nThe crownless again shall be king.\n'")

In [11]:
prompt_template_list = []

In [12]:
for i in range(len(quotes)):
    prompt_template_list.extend(prompt_template.invoke(quotes[0][i]))

In [13]:
prompt_template_list[0][1]

"Summarize the following quote in simple English in a paragraph: 'All that is gold does not glitter,\nNot all those who wander are lost;\nThe old that is strong does not wither,\nDeep roots are not reached by the frost.\n\nFrom the ashes a fire shall be woken,\nA light from the shadows shall spring;\nRenewed shall be blade that was broken,\nThe crownless again shall be king.\n'"

### Confusion over iterating in `prompt_template_list`:

* [0][0]: 'text'
* [0][1]: 'The actual quote'
* [1][0]: 'type'
* [1][1]: 'StringPromptValue'

#### Trying a different approach: Appending the prompt individually to the quote using `quotes`

In [44]:
prompt_template_text = []

In [45]:
for i in range(len(quotes)):
    prompt_template_text.append(f"Summarize the following quote in simple English in a paragraph: {quotes[0][i]}")

In [48]:
len(prompt_template_text), type(prompt_template_text)

(2994, list)

In [55]:
type(quotes)

pandas.core.frame.DataFrame

In [58]:
quotes["prompt_template"] = prompt_template_text
quotes.rename(columns={0:"quote"}, inplace=True)
quotes

,quote,prompt_template
0,"All that is gold does not glitter,\nNot all th...",Summarize the following quote in simple Englis...
1,Not all those who wander are lost.\n,Summarize the following quote in simple Englis...
2,"I wish it need not have happened in my time,"" ...",Summarize the following quote in simple Englis...
3,I don't know half of you half as well as I sho...,Summarize the following quote in simple Englis...
4,All we have to decide is what to do with the t...,Summarize the following quote in simple Englis...
...,...,...
2989,He that breaks a thing to find out what it is ...,Summarize the following quote in simple Englis...
2990,"Home is behind, the world ahead,\nAnd there ar...",Summarize the following quote in simple Englis...
2991,"There is more in you of good than you know, ch...",Summarize the following quote in simple Englis...
2992,"Good Morning!"" said Bilbo, and he meant it. Th...",Summarize the following quote in simple Englis...


In [68]:
quotes.to_json("quotes_with_prompt.json", orient="index")

### Adding Summaries to `quotes_with_prompt.json` with Gemini

In [16]:
quotes = pd.read_json("quotes_with_prompt.json", lines=True).T
quotes

,0
0,"{'quote': 'All that is gold does not glitter, ..."
1,{'quote': 'Not all those who wander are lost. ...
2,{'quote': 'I wish it need not have happened in...
3,{'quote': 'I don't know half of you half as we...
4,{'quote': 'All we have to decide is what to do...
...,...
2989,{'quote': 'He that breaks a thing to find out ...
2990,"{'quote': 'Home is behind, the world ahead, An..."
2991,{'quote': 'There is more in you of good than y...
2992,"{'quote': 'Good Morning!"" said Bilbo, and he m..."


In [14]:
llm = GoogleGenerativeAI(model="gemini-2.5-flash")

response = llm.invoke("Summarize the gist of the quote in a paragraph in simple English: 'Not all those who wander are lost.'")

In [15]:
display(Markdown(response))

The quote "Not all those who wander are lost" means that someone who appears to be moving without a clear, set path or aimlessly exploring isn't necessarily confused, without purpose, or unable to find their way. Instead, it suggests that sometimes, taking an unconventional route or simply enjoying the journey without a strict destination can be a deliberate and meaningful choice. These individuals might be discovering new things, finding their own unique direction, or intentionally seeking experiences, all while having an internal sense of purpose, even if their path isn't obvious to others.

In [22]:
summaries = []

In [25]:
for i in tqdm(range(len(quotes))):
    generated_summary = llm.invoke(quotes[0][i]["prompt_template"])
    summaries.append(generated_summary)
    time.sleep(2)

  7%|▋         | 203/2994 [23:08<5:01:05,  6.47s/it]Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 250
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 54
}
].
  7%|▋         | 204/2994 [23:17<5:28:11,  7.06s/it]Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds a

ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 250
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 32
}
]

In [90]:
# deal with first 200 summaries at the moment
summaries = summaries[:200]
summaries

["The speaker is telling a group of people that he hasn't gotten to know many of them as well as he would have liked. He then adds, somewhat humbly, that there are others in the group whom he doesn't appreciate or like as much as they genuinely deserve, suggesting that his own feelings fall short of their actual good qualities.",
 "This quote means that the most important choice we have to make in life isn't about external events or what happens *to* us, but rather the fundamental decision of how we use the time we are given. It highlights that our lives are ultimately shaped by the conscious choices we make every day about how to spend our moments, hours, and years. Essentially, it boils down to recognizing that our main task is to decide what we do with our limited time, as this defines our path and experiences.",
 'This quote means that if we prioritized simple joys and shared experiences, like enjoying good food, laughter, and music together, more than we value accumulating money a

In [89]:
type(quotes), type(quotes[0]), type(summaries)

(pandas.core.frame.DataFrame, pandas.core.series.Series, list)

In [91]:
summaries[0]

"The speaker is telling a group of people that he hasn't gotten to know many of them as well as he would have liked. He then adds, somewhat humbly, that there are others in the group whom he doesn't appreciate or like as much as they genuinely deserve, suggesting that his own feelings fall short of their actual good qualities."

In [94]:
# quote and summary mismatch
quotes[0][0]["quote"], summaries[0]

('All that is gold does not glitter,\nNot all those who wander are lost;\nThe old that is strong does not wither,\nDeep roots are not reached by the frost.\n\nFrom the ashes a fire shall be woken,\nA light from the shadows shall spring;\nRenewed shall be blade that was broken,\nThe crownless again shall be king.\n',
 "The speaker is telling a group of people that he hasn't gotten to know many of them as well as he would have liked. He then adds, somewhat humbly, that there are others in the group whom he doesn't appreciate or like as much as they genuinely deserve, suggesting that his own feelings fall short of their actual good qualities.")

In [34]:
os.mkdir("summaries")